In [ ]:


import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm
import yfinance as yf

def indicator_function(condition):
    return 1 if condition else 0

def calculate_variance_swap_strike(S, r, T, options_data):
    # Extract call and put prices
    call_data = options_data[options_data['Type'] == 'call'].copy()
    put_data = options_data[options_data['Type'] == 'put'].copy()
    
    # Interpolate call and put prices
    strikes = np.unique(options_data['Strike'])
    call_interp = interp1d(call_data['Strike'], call_data['Price'], fill_value="extrapolate")
    put_interp = interp1d(put_data['Strike'], put_data['Price'], fill_value="extrapolate")
    
    # Integrate using numerical methods (trapezoidal rule)
    
    K_min, K_max = strikes.min(), strikes.max()
    K = np.linspace(K_min, S*np.exp(r*T), 500)
    integral1 = np.trapezoid(put_interp(K) / K**2, K)
    K = np.linspace(S*np.exp(r*T),K_max, 500)
    integral2 = np.trapezoid(call_interp(K) / K**2, K)
    
    # Calculate the variance swap strike
    variance_swap_strike = np.sqrt(2 * (integral1+integral2) / T)
    return variance_swap_strike

# Example usage
ticker = "AAPL"
S = 150.0  # Current stock price
r = 0.01  # Risk-free rate
T = 0.5  # Time to maturity (6 months)

# Fetch options data from Yahoo Finance
stock = yf.Ticker(ticker)
expiry = stock.options[0]
opt_chain = stock.option_chain(expiry)
calls = opt_chain.calls[['strike', 'lastPrice']].copy()
puts = opt_chain.puts[['strike', 'lastPrice']].copy()
calls.columns = ['Strike', 'Price']
puts.columns = ['Strike', 'Price']
calls['Type'] = 'call'
puts['Type'] = 'put'
options_data = pd.concat([calls, puts])

# Calculate the variance swap strike
variance_swap_strike = calculate_variance_swap_strike(S, r, T, options_data)
print(f"Variance Swap Strike: {variance_swap_strike:.4f}")